### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

!airflow db init

In [ ]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Запуск шедулера
!airflow scheduler -D

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken  # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http 18273 > /dev/null &

### Задача на разработку

Необходимо реализовать 

- Сенсор который будет возвращать True с вероятностью 0.3 и False с 0.7.
- Написать автогенерацию тасков сенсеров из прошлого пункта, 3 штуки, должны запускаться параллельно.

In [20]:
## ВАШЕ РЕШЕНИЕ
from airflow.sensors.base import BaseSensorOperator
from airflow import DAG
import numpy as np
from datetime import datetime, timedelta
from airflow.utils.dates import days_ago


class CustomSensor(BaseSensorOperator):

    def poke(self, context):
        return_value = np.random.binomial(1, 0.3)
        return bool(return_value)

dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))
args = {
    "poke_interval": 4,
    "timeout": 50,
    "mode": "reschedule",
    "soft_fail": True
}

tasks_list=list()
for i in range(0, 3):
    tasks_list.append(CustomSensor(task_id=f'task_{i}', dag=dag, **args))